<a href="https://www.kaggle.com/code/mafiosoquasar/fake-scene-classification?scriptVersionId=207489595" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import argparse
import os,cv2,math
import random,numpy,pandas
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

In [2]:
import argparse
import os
import random,numpy
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torchvision.models.feature_extraction import create_feature_extractor
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [3]:
seed = 999
print("Random Seed: ", seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

Random Seed:  999


In [4]:
workers = 2
batch_size = 10
nz = 100
num_epochs = 5
lr = 0.0001
beta1 = 0.5
ngpu=1
ngf,nc = 3,3
ndf = 64

transform = transforms.Compose([
    transforms.Resize(400),
    transforms.RandomRotation(15),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

trainset = torchvision.datasets.ImageFolder(root=f'/kaggle/input/fake-scene-dataset/train',transform=transform)
train,test = torch.utils.data.random_split(trainset, [len(trainset)-100,100]) 

dataloader = torch.utils.data.DataLoader(train,batch_size=batch_size,shuffle=True,num_workers=2)
testloader = torch.utils.data.DataLoader(test,batch_size=batch_size,shuffle=True,num_workers=2)

#print(len(dataloader),len(testloader))
device = torch.device("cuda" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [5]:
class EffnetModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        effnet = torchvision.models.efficientnet_v2_m(weights=torchvision.models.efficientnet.EfficientNet_V2_M_Weights.DEFAULT)
        self.model = create_feature_extractor(effnet, ['flatten'])
        self.nn_fracture = torch.nn.Sequential(
            torch.nn.Linear(1280, 2),
        )
    def forward(self, x):
        x = self.model(x)['flatten']
        x = self.nn_fracture(x)
        return x,torch.argmax(x, dim=1)

In [6]:
EFF_NET = EffnetModel().float()
EFF_NET= nn.DataParallel(EFF_NET).to(device)
#EFF_NET
#EFF_NET.load_state_dict(torch.load(f"/kaggle/input/fsc-dataset/8.072827768046409e-05 88.0.pth",weights_only=False,map_location=torch.device('cpu')))

Downloading: "https://download.pytorch.org/models/efficientnet_v2_m-dc08266a.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_m-dc08266a.pth
100%|██████████| 208M/208M [00:01<00:00, 200MB/s]  


In [7]:
criterion = nn.BCEWithLogitsLoss()

# CrossEntropyLoss
# MSELoss
# L1Loss
# BCELoss
# BCEWithLogitsLoss

optimizer = optim.AdamW(EFF_NET.parameters(), lr=lr, betas=(beta1, 0.999))
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.86)

In [ ]:
print(train.dataset.classes)
z_w_=[]
high_rf,i_w,z_k=0,0,0
high_rf_=1

lab={
    'editada' : 0,
    'real' : 1
}
fsc_submission=pandas.read_csv("/kaggle/input/cidaut-ai-fake-scene-classification-2024/sample_submission.csv", index_col ="image")#.to_dict('list')

while(i_w<4000):
    z_,z,z_w=0,0,{}
    correct=0
    for i, data in enumerate(dataloader, 0):
        real=data[0].to(device)
        label=data[1].to(device) #
        #print(data[0].shape)
        raw_label=torch.from_numpy(numpy.array([[0,1] if i==1 else [1,0] for i in data[1] ])).float().to(device)
        raw_output,output = EFF_NET(real)
        raw_output,output = raw_output.float() ,torch.tensor([lab[train.dataset.classes[i]] for i in output.view(-1)])
        #print(raw_label,raw_output)
        err_real = criterion(raw_output,raw_label) #err_real.requires_grad = True
        optimizer.zero_grad()
        err_real.backward()
        optimizer.step()
        
    for i, data in enumerate(testloader, 0):
        real=data[0].to(device)
        label=data[1]
        raw_output,output = EFF_NET(real)
        output=torch.tensor([lab[train.dataset.classes[i]] for i in output.view(-1)])
        z_+=len(output)
        correct+=(output==label).float().sum()

    print(output,label)
        
    acc=100*correct/z_
    z_w_.append(acc)
    
    print(f"EPOCH: {z_k} LOSS_FSC: {err_real.item()} ACCURACY: {acc}")
    #print(z_)	
        
    if len(z_w_)>=100:
        if len([True for i in range(1,4) if z_w_[len(z_w_)-i]<=z_w_[len(z_w_)-3] and z_w_[len(z_w_)-i]>=z_w_[len(z_w_)-4]])==3:
            z_w_=[]
            print(optimizer.param_groups[0]["lr"])
            scheduler.step()
            print(optimizer.param_groups[0]["lr"])
    
    if err_real.item()<=0.005:
        torch.save(EFF_NET.state_dict(),f'/kaggle/working/{err_real.item()} {z_k}.pth')# {z_add}{acc}
    
    if z_k==300:
        break
    z_k+=1
    i_w+=1
    

['editada', 'real']


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):


tensor([1, 1, 1, 0, 0, 0, 0, 1, 0, 1]) tensor([1, 1, 1, 0, 0, 0, 0, 1, 0, 1])
EPOCH: 0 LOSS_FSC: 0.13960282504558563 ACCURACY: 86.0
tensor([1, 1, 1, 1, 0, 1, 1, 0, 1, 0]) tensor([1, 1, 1, 1, 0, 1, 1, 0, 1, 0])
EPOCH: 1 LOSS_FSC: 0.06961661577224731 ACCURACY: 90.0
tensor([1, 0, 0, 1, 1, 1, 1, 0, 1, 0]) tensor([1, 0, 0, 1, 1, 1, 1, 0, 1, 1])
EPOCH: 2 LOSS_FSC: 0.6192228198051453 ACCURACY: 92.0
tensor([1, 0, 1, 0, 1, 0, 0, 0, 1, 1]) tensor([1, 0, 1, 1, 1, 0, 0, 0, 1, 1])
EPOCH: 3 LOSS_FSC: 0.11520235985517502 ACCURACY: 93.0
tensor([0, 1, 0, 1, 1, 1, 0, 1, 1, 0]) tensor([0, 1, 1, 1, 1, 1, 0, 1, 1, 0])
EPOCH: 4 LOSS_FSC: 0.056447505950927734 ACCURACY: 95.0
tensor([0, 1, 0, 0, 1, 1, 0, 1, 1, 1]) tensor([0, 1, 0, 0, 1, 1, 0, 1, 1, 1])
EPOCH: 5 LOSS_FSC: 0.01117036584764719 ACCURACY: 98.0
tensor([0, 1, 0, 0, 1, 1, 0, 1, 1, 1]) tensor([0, 0, 0, 0, 0, 1, 0, 1, 1, 1])
EPOCH: 6 LOSS_FSC: 0.010688229463994503 ACCURACY: 94.0
tensor([0, 0, 0, 0, 1, 1, 1, 0, 0, 1]) tensor([0, 0, 0, 0, 1, 1, 1, 0, 0, 1